In [1]:
library(tidyverse)

source("../../evaluation_utils/plots_eda.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: viridisLite



May take several hours to run. You can skip this step and go to the `evaluation` folder to see the results.

# Load and central correction

In [ ]:
path_to_data <- "./"

n_runs = 1

In [ ]:
for(mode in c(
    "balanced",
    "mild_imbalanced", "strong_imbalanced"
)){
    # if folder does not exist, create it
    if(!dir.exists(paste0(path_to_data, mode, "/after/runs"))){
        dir.create(paste0(path_to_data, mode, "/after/runs"), recursive = T)
    }

    print(paste0("Processing mode: ", mode))

    metadata <- read.csv(paste0(path_to_data, mode, "/all_metadata.tsv"), sep = "\t") %>%
        as.data.frame()

    for(j in 1:n_runs){
        intensities <- read.csv(paste0(path_to_data, mode, "/before/intermediate/", j, "_intensities_data.tsv"), sep = "\t") %>%
            as.data.frame() %>%
            column_to_rownames("rowname")

        metadata <- metadata %>%
          mutate(condition = as.factor(condition), lab = as.factor(lab))

        design <- model.matrix(~ condition, metadata)
        colnames(design) <- c("Intercept", "condition")

        intensities_corrected <- limma::removeBatchEffect(
                intensities[,metadata$file],
                metadata$lab,
                design = design) %>%
            as.data.frame()

        # write to file
        write.table(intensities_corrected %>% rownames_to_column("rowname"),
                    paste0(path_to_data, mode, "/after/runs/", j, "_R_corrected.tsv"),
                    sep = "\t", row.names = F)

        print(paste0("\t\tSaved corrected intensities for mode: ", mode))
    }
}

[1] "Processing mode: balanced"
[1] "\t\tSaved corrected intensities for mode: balanced"


Warning message in file(file, "rt"):
“cannot open file './balanced/before/intermediate/2_intensities_data.tsv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


# Session info

In [4]:
print(sessionInfo())

R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: Ubuntu 24.04.2 LTS

Matrix products: default
BLAS/LAPACK: /home/yuliya-cosybio/miniforge3/envs/fedRBE/lib/libopenblasp-r0.3.30.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] viridis_0.6.5     viridisLite_0.4.2 ggsci_4.2.0       umap_0.2.10.0    
 [5] patchwork_1.3.2   gridExtra_2.3     lubridate_1.9.4   forcats_1.0.1    
 [9] stringr_1.6.0     dplyr_1.1.4       purrr_1.2.0     